In [ ]:
#| default_exp outorgadas
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path


In [ ]:

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Anatel

> Este módulo consolida as bases da Anatel e realiza pós-processamento dos dados obtidos.

In [9]:
#| export
import os
from typing import List, Union

import numpy as np
import pandas as pd
from dotenv import find_dotenv, load_dotenv

from extracao.datasources.sitarweb import Stel, Radcom, SQLSERVER_PARAMS
from extracao.datasources.mosaico import MONGO_URI
from extracao.datasources.srd import SRD
from extracao.datasources.telecom import Telecom
from extracao.datasources.smp import SMP

In [6]:
#| export
load_dotenv(find_dotenv())

True

## Base Consolidada ANATEL

In [ ]:
class Outorgadas:
    

In [ ]:
def validar_coords(
    row: pd.Series,  # Linha de um DataFrame
    connector: pyodbc.Connection = None,  # Conector de Banco de Dados
) -> List[str]:  # DataFrame com dados do município
    """Valida os dados de coordenadas e município em `row` no polígono dos municípios em banco corporativ do IBGE"""

    mun, cod, lat, long = (
        row.Município,
        row.Código_Município,
        row.Latitude,
        row.Longitude,
    )
    is_valid = "-1"
    conn = connect_db() if connector is None else connector
    crsr = conn.cursor()
    sql = SQL_VALIDA_COORD.format(long, lat, cod)
    crsr.execute(sql)
    result = crsr.fetchone()
    if result is not None:
        mun = result.NO_MUNICIPIO
        lat = result.NU_LATITUDE
        long = result.NU_LONGITUDE
        is_valid = result.COORD_VALIDA
    if connector is None:
        del conn
    return [str(mun), str(lat), str(long), str(is_valid)]

In [ ]:
def update_cached_df(df: pd.DataFrame, df_cache: pd.DataFrame) -> pd.DataFrame:
    """Mescla ambos dataframes eliminando os excluídos (existentes somente em df_cache)"""

    # Merge dataframes based on all columns except "Coords_Valida_IBGE"
    merged = pd.merge(
        df_cache,
        df,
        on=list(df.columns),
        how="outer",
        indicator=True,
        copy=False,
        validate="one_to_one",
    ).astype("string")

    # Exclude rows only present in df_cache
    df_cache = merged[merged["_merge"] != "left_only"]

    # inplace=True not working
    df_cache.loc[:, ["Latitude", "Longitude"]] = df_cache.loc[
        :, ["Latitude", "Longitude"]
    ].fillna("-1")

    # # Drop the _merge column
    return df_cache.drop(columns="_merge")

In [ ]:
def _validar_coords_base(
    df: pd.DataFrame,  # DataFrame com os dados da Anatel
    df_cache: pd.DataFrame,  # DataFrame validado anteriormente, usado como cache
    connector: pyodbc.Connection = None,  # Conector de Banco de Dados
) -> pd.DataFrame:  # DataFrame com as coordenadas validadas na base do IBGE
    """Valida as coordenadas consultado a Base Corporativa do IBGE, excluindo o que já está no cache na versão anterior"""

    df_cache = update_cached_df(df.astype("string"), df_cache.astype("string"))

    municipios = pd.read_csv(
        Path(__file__).parent / "arquivos" / "municipios_coordenadas.csv",
        usecols=["codigo_ibge"],
        dtype="string",
    )

    # valida os códigos de municípios
    valid_cod_mun = df_cache.Código_Município.isin(municipios.codigo_ibge)

    df_cache.loc[~valid_cod_mun, "Coords_Valida_IBGE"] = "-1"

    subset = df_cache.Coords_Valida_IBGE.isna()

    if linhas := list(
        df_cache.loc[
            subset, ["Município", "Código_Município", "Latitude", "Longitude"]
        ].itertuples()
    ):
        func = partialler(validar_coords, connector=connector)

        # Gambiarra para evitar compartilhamento da mesma conexão de banco em diferentes threads
        n_workers = 1 if connector is not None else 20

        ibge = [
            "Município_IBGE",
            "Latitude_IBGE",
            "Longitude_IBGE",
            "Coords_Valida_IBGE",
        ]

        df_cache.loc[subset, ibge] = parallel(
            func, linhas, threadpool=True, n_workers=n_workers, progress=True
        )

    df_cache = df_cache.astype("string")

    df_cache.loc[df_cache.Coords_Valida_IBGE == "-1", "Coords_Valida_IBGE"] = pd.NA

    return df_cache

In [ ]:
def update_base(
    conn: pyodbc.Connection,  # Objeto de conexão de banco
    clientMongoDB: MongoClient,  # Objeto de conexão com o MongoDB
    folder: Union[str, Path],  # Pasta onde salvar os arquivos
    conn_threads: bool = False,  # Flag para criar uma conexão de banco por thread
) -> pd.DataFrame:  # DataFrame com os dados atualizados
    # sourcery skip: use-fstring-for-concatenation
    """Wrapper que atualiza opcionalmente lê e atualiza as 4 bases indicadas anteriormente, as combina e salva o arquivo consolidado na folder `folder`"""
    stel = update_stel(conn, folder)
    radcom = update_radcom(conn, folder)
    mosaico = update_srd(clientMongoDB, folder)
    telecom = update_telecom(clientMongoDB, folder)

    df = (
        pd.concat([mosaico, radcom, stel, telecom])
        .sort_values(["Frequência", "Latitude", "Longitude"])
        .reset_index(drop=True)
    ).astype("string")

    # inplace not working!
    df.loc[:, ["Latitude", "Longitude"]] = df.loc[:, ["Latitude", "Longitude"]].fillna(
        "0"
    )
    try:
        df_cache = _read_df(folder, "base")
    except FileNotFoundError:
        df_cache = pd.DataFrame(columns=df.columns.to_list() + ["Coords_Valida_IBGE"])

    connector = None if conn_threads else conn

    df_cache = _validar_coords_base(df, df_cache, connector)

    return _save_df(df_cache, folder, "base")